## Importing Library

In [1]:
import cv2
import mediapipe as mp
import math
import serial.tools.list_ports
import time

## Connecting To theSerial Monitor

In [2]:
ports = serial.tools.list_ports.comports()
serialInst = serial.Serial()


for x in range(len(ports)):
    if(ports[x][1][0:7] == "Arduino"):
        portvar = "COM" + str(ports[0][0][3])
        print("Connecting To Serial Monitor Of: ", portvar)
        
serialInst.baudraten = 9600
serialInst.port = portvar
serialInst.open()

Connecting To Serial Monitor Of:  COM5


## Command Sending Function

In [3]:
def LoobCommand():
    while True:
        command = input("Arduino command ON/OFF: ")
        serialInst.write(command.encode('utf-8'))
    
        if command == "exit":
            break
            
def Command(stg):
    command = str(stg)
    serialInst.write(command.encode('utf-8'))

In [4]:
def TurnAll():
    for i in range(10):
        Command(str(i) + "1")
        time.sleep(1.2)
        
def CloseAll():
    for i in range(10):
        Command(str(i) + "0")
        time.sleep(1.2)


### Finger Distance Functions:

In [5]:
def DrawFinger(landmarks, FingerTip):
        # Get the coordinates of landmarks
        x, y = landmarks[FingerTip].x, landmarks[FingerTip].y
        # Draw circles at the landmark positions
        h, w, c = img.shape
        cx, cy = int(x * w), int(y * h)
        # this part to draw the hand body
        cv2.circle(img, (cx, cy), 10, (255,255,255), thickness=2)
        return int(cx), int(cy)

def distance_between_points(x1, y1, x2, y2):
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

## Video Capture

In [6]:
cap = cv2.VideoCapture(0)

mpHands = mp.solutions.hands
hands = mpHands.Hands()
mpDraw = mp.solutions.drawing_utils

while True:
    success, img = cap.read()
    img = cv2.flip(img,1)
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)
    
    # Getting the land mark
    if results.multi_hand_landmarks:
        for handLms in results.multi_hand_landmarks:
            for id, lm in enumerate(handLms.landmark):
                h, w, c = img.shape
                cx, cy = int(lm.x * w), int(lm.y * h)
        landmarks = handLms.landmark
                
        # Drawing the index finger
        cx8,cy8 = DrawFinger(landmarks,8)
        cx4,cy4 = DrawFinger(landmarks,4)
        cx0,cy0 = DrawFinger(landmarks,0)
        
        cv2.line(img , (cx4,cy4) , (cx8,cy8) , (255,0,0) , 1)
        cv2.line(img , (cx0,cy0) , (cx8,cy8) , (0,255,0) , 1)
        cv2.line(img , (cx4,cy4) , (cx0,cy0) , (0,0,255) , 1)
        
        e2f = int(distance_between_points(cx8,cy8,cx4,cy4))
        z2f = int(distance_between_points(cx0,cy0,cx4,cy4))
        e2z = int(distance_between_points(cx8,cy8,cx0,cy0))
  
        B = (z2f*z2f +  e2z * e2z - e2f*e2f) / (2 * z2f * e2z)
        cv2.putText(img, str(B*10 - 1) , (500, 400), cv2.FONT_HERSHEY_SIMPLEX, 3, (255,255, 255), 2)
        Command(str(int(B*10 - 1)))
    
        time.sleep(1)
    cv2.imshow("Image", img)
    
    # Check for the 'Esc' key press
    key = cv2.waitKey(1)
    if key == 27:  # 27 is the keycode for 'Esc'
        break

cap.release()
cv2.destroyAllWindows()